In [1]:
import gradio as gr
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go


calorie_data = []
bmi_history = []

def calculate_bmi(weight, height):
    """Calculate BMI and return category"""
    if weight <= 0 or height <= 0:
        return "Please enter valid positive numbers", ""

    height_m = height / 100
    bmi = weight / (height_m ** 2)

    if bmi < 18.5:
        category = "Underweight"
        color = "#5E60EA"
    elif 18.5 <= bmi < 25:
        category = "Normal weight"
        color = "#14B8A6"
    elif 25 <= bmi < 30:
        category = "Overweight"
        color = "#D4C92D"
    else:
        category = "Obese"
        color = "#94160D"

    result = f"<div style='text-align: center; padding: 20px;'>"
    result += f"<h2 style='color: {color}; font-size: 48px; margin: 10px 0;'>{bmi:.1f}</h2>"
    result += f"<h3 style='color: {color}; margin: 5px 0;'>{category}</h3>"
    result += f"</div>"

    return result

def add_bmi_record(weight, height, record_date):
    """Add BMI record to history"""
    global bmi_history

    try:
        if weight <= 0 or height <= 0:
            return None, "<p style='color: red;'>Please enter valid weight and height</p>"

        height_m = height / 100
        bmi = weight / (height_m ** 2)

        if record_date:
            date = datetime.strptime(record_date, "%Y-%m-%d").date()
        else:
            date = datetime.now().date()

        entry = {
            'date': date,
            'weight': weight,
            'height': height,
            'bmi': bmi
        }

        bmi_history = [e for e in bmi_history if e['date'] != date]
        bmi_history.append(entry)
        bmi_history.sort(key=lambda x: x['date'])

        print(f"Added BMI record: {entry}")
        print(f"Total BMI records: {len(bmi_history)}")

        fig = create_bmi_graph()
        history = create_bmi_history_table()

        return fig, history
    except Exception as e:
        print(f"Error in add_bmi_record: {e}")
        import traceback
        traceback.print_exc()
        return None, f"<p style='color: red;'>Error: {str(e)}</p>"

def create_bmi_graph():
    """Create 12-month BMI line graph"""
    try:
        if not bmi_history:
            fig = go.Figure()
            fig.add_annotation(
                text="No BMI records yet. Start tracking your BMI!",
                xref="paper", yref="paper",
                x=0.5, y=0.5, showarrow=False,
                font=dict(size=16, color="#666666")
            )
            fig.update_layout(
                height=400,
                margin=dict(t=40, b=40, l=40, r=40),
                paper_bgcolor='white',
                plot_bgcolor='#F0FDFA'
            )
            return fig

        dates = [e['date'] for e in bmi_history]
        bmis = [e['bmi'] for e in bmi_history]

        fig = go.Figure()

        fig.add_trace(go.Scatter(
            x=dates,
            y=bmis,
            mode='lines+markers',
            name='BMI',
            line=dict(color='#14B8A6', width=3),
            marker=dict(size=8, color='#14B8A6', line=dict(color='white', width=2))
        ))

        fig.add_hline(y=18.5, line_dash="dash", line_color="#605EEA",
                      annotation_text="Underweight", annotation_position="right")
        fig.add_hline(y=25, line_dash="dash", line_color="#2DD4BF",
                      annotation_text="Overweight", annotation_position="right")
        fig.add_hline(y=30, line_dash="dash", line_color="#94920D",
                      annotation_text="Obese", annotation_position="right")

        fig.add_hrect(y0=18.5, y1=25, fillcolor="#CCFBF1", opacity=0.2, line_width=0)

        fig.update_layout(
            title='BMI Progress (Last 12 Months)',
            xaxis_title='Date',
            yaxis_title='BMI',
            height=450,
            showlegend=True,
            margin=dict(t=60, b=60, l=60, r=100),
            paper_bgcolor='white',
            plot_bgcolor='#F0FDFA',
            font=dict(color='#0F766E'),
            hovermode='x unified'
        )

        return fig
    except Exception as e:
        print(f"Error in create_bmi_graph: {e}")
        import traceback
        traceback.print_exc()
        fig = go.Figure()
        fig.add_annotation(
            text=f"Error creating graph: {str(e)}",
            xref="paper", yref="paper",
            x=0.5, y=0.5, showarrow=False
        )
        return fig

def create_bmi_history_table():
    """Create HTML table of BMI history"""
    try:
        if not bmi_history:
            return "<p style='text-align: center; color: #666; padding: 20px;'>No BMI records yet</p>"

        recent_records = bmi_history[-10:]
        html = "<div style='padding: 10px;'>"
        html += "<h3 style='text-align: center; margin-bottom: 20px; color: #0F766E;'>BMI History</h3>"
        html += "<table style='width: 100%; border-collapse: collapse;'>"
        html += "<tr style='background: #CCFBF1;'>"
        html += "<th style='padding: 12px; text-align: left; border-bottom: 2px solid #5EEAD4; color: #0F766E;'>Date</th>"
        html += "<th style='padding: 12px; text-align: center; border-bottom: 2px solid #5EEAD4; color: #0F766E;'>Weight (kg)</th>"
        html += "<th style='padding: 12px; text-align: center; border-bottom: 2px solid #5EEAD4; color: #0F766E;'>Height (cm)</th>"
        html += "<th style='padding: 12px; text-align: center; border-bottom: 2px solid #5EEAD4; color: #0F766E;'>BMI</th>"
        html += "<th style='padding: 12px; text-align: center; border-bottom: 2px solid #5EEAD4; color: #0F766E;'>Category</th>"
        html += "</tr>"

        for entry in reversed(recent_records):
            bmi = entry['bmi']
            if bmi < 18.5:
                category = "Underweight"
                cat_color = "#5E60EA"
            elif 18.5 <= bmi < 25:
                category = "Normal"
                cat_color = "#14B8A6"
            elif 25 <= bmi < 30:
                category = "Overweight"
                cat_color = "#D4C92D"
            else:
                category = "Obese"
                cat_color = "#94160D"

            html += "<tr style='border-bottom: 1px solid #CCFBF1;'>"
            html += f"<td style='padding: 10px; color: #0F766E;'>{entry['date'].strftime('%Y-%m-%d')}</td>"
            html += f"<td style='padding: 10px; text-align: center; color: #0F766E;'>{entry['weight']:.1f}</td>"
            html += f"<td style='padding: 10px; text-align: center; color: #0F766E;'>{entry['height']:.1f}</td>"
            html += f"<td style='padding: 10px; text-align: center; font-weight: bold; color: #14B8A6;'>{bmi:.1f}</td>"
            html += f"<td style='padding: 10px; text-align: center; font-weight: bold; color: {cat_color};'>{category}</td>"
            html += "</tr>"

        html += "</table></div>"
        return html
    except Exception as e:
        print(f"Error in create_bmi_history_table: {e}")
        import traceback
        traceback.print_exc()
        return f"<p style='color: red;'>Error creating table: {str(e)}</p>"

def calculate_bmr_tdee(weight, height, age, gender, activity):
    """Calculate BMR and TDEE"""
    if weight <= 0 or height <= 0 or age <= 0:
        return "Please enter valid positive numbers"

    if gender == "Male":
        bmr = 10 * weight + 6.25 * height - 5 * age + 5
    else:
        bmr = 10 * weight + 6.25 * height - 5 * age - 161

    activity_multipliers = {
        "Sedentary (little or no exercise)": 1.2,
        "Lightly active (exercise 1-3 days/week)": 1.375,
        "Moderately active (exercise 3-5 days/week)": 1.55,
        "Very active (exercise 6-7 days/week)": 1.725,
        "Super active (physical job or training twice/day)": 1.9
    }

    tdee = bmr * activity_multipliers[activity]

    result = f"<div style='text-align: center; padding: 20px;'>"
    result += f"<div style='margin: 20px 0;'>"
    result += f"<h3 style='color: #14B8A6; margin: 5px 0;'>Basal Metabolic Rate (BMR)</h3>"
    result += f"<h2 style='color: #14B8A6; font-size: 42px; margin: 10px 0;'>{bmr:.0f} cal/day</h2>"
    result += f"</div>"
    result += f"<div style='margin: 20px 0;'>"
    result += f"<h3 style='color: #2DD4BF; margin: 5px 0;'>Total Daily Energy Expenditure (TDEE)</h3>"
    result += f"<h2 style='color: #2DD4BF; font-size: 42px; margin: 10px 0;'>{tdee:.0f} cal/day</h2>"
    result += f"</div>"
    result += f"<div style='margin-top: 30px; padding: 15px; background: rgba(94, 234, 212, 0.1); border-radius: 10px; border: 1px solid #5EEAD4;'>"
    result += f"<p style='color: #0F766E; margin: 5px 0;'><strong>Weight Loss:</strong> {tdee - 500:.0f} cal/day (-0.5 kg/week)</p>"
    result += f"<p style='color: #0F766E; margin: 5px 0;'><strong>Maintain Weight:</strong> {tdee:.0f} cal/day</p>"
    result += f"<p style='color: #0F766E; margin: 5px 0;'><strong>Weight Gain:</strong> {tdee + 500:.0f} cal/day (+0.5 kg/week)</p>"
    result += f"</div>"
    result += f"</div>"

    return result

def add_calorie_entry(food_name, calories, entry_date):
    """Add a calorie entry and return updated graph and history"""
    global calorie_data

    try:
        if not food_name or not food_name.strip():
            return create_calorie_graph(), "<p style='color: red;'>Please enter a food name</p>"

        if calories <= 0:
            return create_calorie_graph(), "<p style='color: red;'>Please enter calories greater than 0</p>"

        if entry_date:
            date = datetime.strptime(entry_date, "%Y-%m-%d").date()
        else:
            date = datetime.now().date()

        entry = {
            'date': date,
            'food': food_name.strip(),
            'calories': float(calories)
        }
        calorie_data.append(entry)

        print(f"Added entry: {entry}")
        print(f"Total entries: {len(calorie_data)}")

        fig = create_calorie_graph()

        history = create_history_table()

        return fig, history
    except Exception as e:
        print(f"Error in add_calorie_entry: {e}")
        return create_calorie_graph(), f"<p style='color: red;'>Error: {str(e)}</p>"

def create_calorie_graph():
    """Create a 7-day calorie graph"""
    try:
        if not calorie_data:
            fig = go.Figure()
            fig.add_annotation(
                text="No data yet. Start tracking your calories!",
                xref="paper", yref="paper",
                x=0.5, y=0.5, showarrow=False,
                font=dict(size=16, color="#666666")
            )
            fig.update_layout(
                height=400,
                margin=dict(t=40, b=40, l=40, r=40)
            )
            return fig

        today = datetime.now().date()
        last_7_days = [today - timedelta(days=i) for i in range(6, -1, -1)]
        daily_calories = {}
        for date in last_7_days:
            daily_calories[date] = 0

        for entry in calorie_data:
            if entry['date'] in daily_calories:
                daily_calories[entry['date']] += entry['calories']

        dates = list(daily_calories.keys())
        calories = list(daily_calories.values())

        fig = go.Figure()

        fig.add_trace(go.Bar(
            x=[d.strftime('%m/%d') for d in dates],
            y=calories,
            marker=dict(
                color='#5EEAD4',
                line=dict(color='#14B8A6', width=2)
            ),
            text=[f'{c:.0f}' for c in calories],
            textposition='outside'
        ))

        fig.update_layout(
            title='7-Day Calorie Tracker',
            xaxis_title='Date',
            yaxis_title='Calories',
            height=400,
            showlegend=False,
            margin=dict(t=60, b=60, l=60, r=30),
            paper_bgcolor='white',
            plot_bgcolor='#F0FDFA',
            font=dict(color='#0F766E')
        )

        return fig
    except Exception as e:
        print(f"Error in create_calorie_graph: {e}")
        import traceback
        traceback.print_exc()
        fig = go.Figure()
        fig.add_annotation(
            text=f"Error creating graph: {str(e)}",
            xref="paper", yref="paper",
            x=0.5, y=0.5, showarrow=False
        )
        return fig

def create_history_table():
    """Create HTML table of today's entries"""
    try:
        if not calorie_data:
            return "<p style='text-align: center; color: #666; padding: 20px;'>No entries yet</p>"

        today = datetime.now().date()
        today_entries = [e for e in calorie_data if e['date'] == today]

        if not today_entries:
            return "<p style='text-align: center; color: #666; padding: 20px;'>No entries for today</p>"

        total_calories = sum(e['calories'] for e in today_entries)

        html = "<div style='padding: 10px;'>"
        html += f"<h3 style='text-align: center; margin-bottom: 20px; color: #0F766E;'>Today's Total: <span style='color: #14B8A6;'>{total_calories:.0f} cal</span></h3>"
        html += "<table style='width: 100%; border-collapse: collapse;'>"
        html += "<tr style='background: #CCFBF1;'>"
        html += "<th style='padding: 12px; text-align: left; border-bottom: 2px solid #5EEAD4; color: #0F766E;'>Food</th>"
        html += "<th style='padding: 12px; text-align: right; border-bottom: 2px solid #5EEAD4; color: #0F766E;'>Calories</th>"
        html += "</tr>"

        for entry in reversed(today_entries[-10:]):
            html += "<tr style='border-bottom: 1px solid #CCFBF1;'>"
            html += f"<td style='padding: 10px; color: #0F766E;'>{entry['food']}</td>"
            html += f"<td style='padding: 10px; text-align: right; font-weight: bold; color: #14B8A6;'>{entry['calories']:.0f}</td>"
            html += "</tr>"

        html += "</table></div>"
        return html
    except Exception as e:
        print(f"Error in create_history_table: {e}")
        import traceback
        traceback.print_exc()
        return f"<p style='color: red;'>Error creating table: {str(e)}</p>"




custom_css = """
@import url('https://fonts.googleapis.com/css2?family=Montserrat:wght@300;400;500;600;700&display=swap');

* {
    font-family: 'Montserrat', sans-serif !important;
}

.gradio-container {
    font-family: 'Montserrat', sans-serif !important;
}

h1, h2, h3, h4, h5, h6 {
    font-family: 'Montserrat', sans-serif !important;
}

label {
    font-weight: 500 !important;
}
"""



with gr.Blocks(theme=gr.themes.Soft(primary_hue="teal", secondary_hue="emerald"), css=custom_css) as app:
    gr.Markdown(
        """
        # Health & Fitness Tracker
        ### Complete your data to analyze your health
        """,
        elem_id="header"
    )

    with gr.Tabs():
        with gr.Tab("BMI Calculator & Tracker"):
            gr.Markdown("## Body Mass Index Calculator & Progress Tracker", elem_id="tab-title")
            gr.Markdown("Calculate your BMI, track your progress over time, and visualize your health journey", elem_id="tab-subtitle")

            with gr.Row():
                with gr.Column(scale=1):
                    bmi_weight = gr.Number(label="Weight (kg)", value=70, minimum=0)
                    bmi_height = gr.Number(label="Height (cm)", value=170, minimum=0)

                    with gr.Row():
                        bmi_calculate = gr.Button("Calculate BMI", variant="secondary", size="lg")
                        track_button = gr.Button("Save to History", variant="primary", size="lg")

                    track_date = gr.Textbox(
                        label="Date (YYYY-MM-DD)",
                        placeholder="Leave empty for today",
                        value=""
                    )

                    bmi_output = gr.HTML(label="Current BMI")

                    with gr.Accordion("BMI History", open=True):
                        bmi_history_output = gr.HTML()

                with gr.Column(scale=2):
                    bmi_graph = gr.Plot(label="12-Month BMI Progress")

            bmi_calculate.click(
                fn=calculate_bmi,
                inputs=[bmi_weight, bmi_height],
                outputs=bmi_output
            )

            track_button.click(
                fn=add_bmi_record,
                inputs=[bmi_weight, bmi_height, track_date],
                outputs=[bmi_graph, bmi_history_output]
            )

            bmi_graph.value = create_bmi_graph()
            bmi_history_output.value = create_bmi_history_table()




        with gr.Tab("BMR & TDEE Calculator"):
            gr.Markdown("## Metabolic Rate Calculator", elem_id="tab-title")
            gr.Markdown("Calculate your daily calorie needs based on your lifestyle", elem_id="tab-subtitle")

            with gr.Row():
                with gr.Column():
                    bmr_weight = gr.Number(label="Weight (kg)", value=70, minimum=0)
                    bmr_height = gr.Number(label="Height (cm)", value=170, minimum=0)
                    bmr_age = gr.Number(label="Age (years)", value=30, minimum=0)
                    bmr_gender = gr.Radio(["Male", "Female"], label="Gender", value="Male")
                    bmr_activity = gr.Dropdown(
                        [
                            "Sedentary (little or no exercise)",
                            "Lightly active (exercise 1-3 days/week)",
                            "Moderately active (exercise 3-5 days/week)",
                            "Very active (exercise 6-7 days/week)",
                            "Super active (physical job or training twice/day)"
                        ],
                        label="Activity Level",
                        value="Moderately active (exercise 3-5 days/week)"
                    )
                    bmr_calculate = gr.Button("Calculate BMR & TDEE", variant="primary", size="lg")

                with gr.Column():
                    bmr_output = gr.HTML(label="Results")

            bmr_calculate.click(
                fn=calculate_bmr_tdee,
                inputs=[bmr_weight, bmr_height, bmr_age, bmr_gender, bmr_activity],
                outputs=bmr_output
            )




        with gr.Tab("Calorie Tracker"):
            gr.Markdown("## Daily Calorie Tracker", elem_id="tab-title")
            gr.Markdown("Track your daily food intake and monitor your 7-day calorie history", elem_id="tab-subtitle")

            with gr.Row():
                with gr.Column(scale=1):
                    food_name = gr.Textbox(label="Food Name", placeholder="e.g., Grilled Chicken")
                    food_calories = gr.Number(label="Calories", value=0, minimum=0)
                    add_button = gr.Button("Add Entry", variant="primary", size="lg")

                    with gr.Accordion("Today's Entries", open=True):
                        history_output = gr.HTML()

                with gr.Column(scale=2):
                    calorie_graph = gr.Plot(label="7-Day Calorie Chart")

            def init_tracker():
                return create_calorie_graph(), create_history_table()

            calorie_graph.value = create_calorie_graph()
            history_output.value = create_history_table()

            add_button.click(
                fn=add_calorie_entry,
                inputs=[food_name, food_calories],
                outputs=[calorie_graph, history_output]
            )



if __name__ == "__main__":
    app.launch(share=True)

/usr/local/lib/python3.12/dist-packages/gradio/utils.py:1052: UserWarning:

Expected 3 arguments for function <function add_calorie_entry at 0x7cdaf2aa4fe0>, received 2.

/usr/local/lib/python3.12/dist-packages/gradio/utils.py:1056: UserWarning:

Expected at least 3 arguments for function <function add_calorie_entry at 0x7cdaf2aa4fe0>, received 2.



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://753204f56e3b4f8770.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
